<a href="https://colab.research.google.com/github/ssandanshi/Speaker-Recognition/blob/master/Speaker_Recognition_HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5889 sha256=09692e138a9ae6c29a062fb0d9d2e16c409222213e923695e7f3dba00db0e432
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [0]:
import os
import glob
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import keras.backend as K
import keras
from python_speech_features import fbank, delta
import operator
from collections import OrderedDict
from multiprocessing import Pool
from time import time
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from shutil import copyfile
import pickle

Using TensorFlow backend.


In [0]:
NUM_FRAMES=399
BATCH_SIZE=32
TRIPLET_PER_BATCH=3

# Test on test set

In [0]:
!cp drive/My\ Drive/SSE/vox_test_npy.zip .
!unzip -q vox_test_npy.zip

In [0]:
!ls

drive  sample_data  vox_test_npy  vox_test_npy.zip


In [0]:
test_catalog = create_catalogue("vox_test_npy","*.npy",flag=True)
test_catalog

Found 4874 files with 40 different speakers.


,filename,class
0,vox_test_npy/id10292_29.npy,id10292
1,vox_test_npy/id10297_53.npy,id10297
2,vox_test_npy/id10276_76.npy,id10276
3,vox_test_npy/id10292_75.npy,id10292
4,vox_test_npy/id10283_14.npy,id10283
...,...,...
4869,vox_test_npy/id10300_258.npy,id10300
4870,vox_test_npy/id10307_148.npy,id10307
4871,vox_test_npy/id10277_60.npy,id10277
4872,vox_test_npy/id10276_23.npy,id10276


In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
le2 = LabelEncoder()
test_catalog["class"] = le2.fit_transform(test_catalog["class"])

In [0]:
le2.classes_

array(['id10270', 'id10271', 'id10272', 'id10273', 'id10274', 'id10275',
       'id10276', 'id10277', 'id10278', 'id10279', 'id10280', 'id10281',
       'id10282', 'id10283', 'id10284', 'id10285', 'id10286', 'id10287',
       'id10288', 'id10289', 'id10290', 'id10291', 'id10292', 'id10293',
       'id10294', 'id10295', 'id10296', 'id10297', 'id10298', 'id10299',
       'id10300', 'id10301', 'id10302', 'id10303', 'id10304', 'id10305',
       'id10306', 'id10307', 'id10308', 'id10309'], dtype=object)

In [0]:
test_catalog

,filename,class
0,vox_test_npy/id10292_29.npy,22
1,vox_test_npy/id10297_53.npy,27
2,vox_test_npy/id10276_76.npy,6
3,vox_test_npy/id10292_75.npy,22
4,vox_test_npy/id10283_14.npy,13
...,...,...
4869,vox_test_npy/id10300_258.npy,30
4870,vox_test_npy/id10307_148.npy,37
4871,vox_test_npy/id10277_60.npy,7
4872,vox_test_npy/id10276_23.npy,6


In [0]:
def split_balanced_and_load_data(data,samples_per_class,model=None,batch_size=10,top_classes=None,seed=999):
  if top_classes is None:
    classes = data["class"].unique()
  else:
    classes = data["class"].value_counts()[:top_classes].index.to_list()
    data = data[data["class"].isin(classes)]
  train = pd.DataFrame()
  removed_indices = []
  for cl in classes:
    data_class = data[data["class"]==cl]
    if samples_per_class<int(len(data_class)/2):
      sample = data_class.sample(samples_per_class,random_state=seed)
      removed_indices.extend(sample.index.to_list())
      train = pd.concat([train,sample], axis=0)
    else:
      sample = data_class.sample(int(len(data_class)/2),random_state=seed)
      removed_indices.extend(sample.index.to_list())
      train = pd.concat([train,sample], axis=0)
  test = data.loc[~data.index.isin(removed_indices)]
  train = train.reset_index(drop=True)
  test = test.reset_index(drop=True)
  trainX, trainY, testX, testY = np.array(train["filename"]),np.array(train["class"]),np.array(test["filename"]),np.array(test["class"])

  if model is not None:
    trainX = predict_model_batch(model,np.array([np.load(x) for x in trainX]),batch_size=batch_size)
    testX = predict_model_batch(model,np.array([np.load(x) for x in testX]),batch_size=batch_size)
  else:
    trainX = np.array([np.load(x) for x in trainX])
    testX = np.array([np.load(x) for x in testX])
    trainX = np.reshape(trainX,(trainX.shape[0],trainX.shape[1]*trainX.shape[2]*trainX.shape[3]))
    testX = np.reshape(testX,(testX.shape[0],testX.shape[1]*testX.shape[2]*testX.shape[3]))
  return trainX, trainY, testX, testY

In [0]:
best_model = None
samples_per_class = 50
num_classes = len(test_catalog["class"].unique())
top_classes = None

In [0]:
trainX, trainY, testX, testY = split_balanced_and_load_data(test_catalog,samples_per_class,batch_size=20,top_classes=top_classes)

In [0]:
print(trainX.shape,trainY.shape,testX.shape,testY.shape)

(1688, 25536) (1688,) (3186, 25536) (3186,)


In [0]:
# print(trainX[0])

In [0]:
! pip install hmmlearn

In [0]:
from hmmlearn import hmm

In [0]:
class Model():
  def __init__(self, CATEGORY=None, n_comp=2, n_mix = 3, cov_type='diag', n_iter=100):
    super(Model, self).__init__()
    self.CATEGORY = CATEGORY
    self.category = len(CATEGORY)
    self.n_comp = n_comp
    self.n_mix = n_mix
    self.cov_type = cov_type
    self.n_iter = n_iter
    self.models = []
    for k in range(self.category):
      model = hmm.GMMHMM(n_components=self.n_comp, n_mix = self.n_mix, 
								covariance_type=self.cov_type, n_iter=self.n_iter)
      self.models.append(model)
  def train(self, train_data = None, train_label = None):
    k_label = dict(zip(range(self.category), np.sort(np.unique(train_label))))
    for k, label in k_label.items():
      subdata = []
      model = self.models[k]
      model.fit(train_data[np.where(train_label==label)[0]])
  def test(self, test_data = None, test_label = None):
    k_label = dict(zip(range(self.category), np.sort(np.unique(test_label))))
    result = []
    for k, label in k_label.items():
      subre = []
      model = self.models[k]
      totalnum = len(test_label)
      for i in range(totalnum):
        re = model.score(test_data[i].reshape(1,-1))
        subre.append(re)
      result.append(subre)
    result = np.vstack(result).argmax(axis=0)
    result = [self.CATEGORY[label] for label in result]
    correctnum = 0
    for i in range(totalnum):
      if result[i] == testY[i]:
        correctnum+=1
    print('accuracy',correctnum/totalnum)

In [0]:
from sklearn.externals import joblib
def save(self, path="models.pkl"):
		joblib.dump(self.models, path)
def load(self, path="models.pkl"):
		self.models = joblib.load(path)

In [0]:
CATEGORY = np.unique(trainY)
models = Model(CATEGORY=CATEGORY)

In [0]:
models.train(train_data = trainX, train_label = trainY)

In [0]:
models.test(test_data=testX, test_label=testY)

accuracy 0.383793410507569
